## Greedy assembly workflow

In [ ]:
r""" Full assembly workflow with a greedy approach.
 _____  _______  _    _ 
|  __ \|__   __|| |  | |
| |  | |  | |   | |  | |
| |  | |  | |   | |  | |
| |__| |  | |   | |__| |
|_____/   |_|   |______|

__authors__ = Marco Reverenna & Konstantinos Kalogeropoulus
__copyright__ = Copyright 2024-2025
__research-group__ = DTU Biosustain (Multi-omics Network Analytics) and DTU Bioengineering
__date__ = 26 Jun 2025
__maintainer__ = Marco Reverenna
__email__ = marcor@dtu.dk
__status__ = Dev
"""

In [ ]:
import os
import sys

script_dir = os.getcwd() 
sys.path.append(os.path.join(script_dir, '../src'))

In [ ]:
# my modules
import greedy_method as greedy
import mapping as map
import consensus as cons
import alignment as align
import clustering as clus
import preprocessing as prep
import compute_statistics as comp_stat

# import libraries
from tqdm import tqdm
from Bio import SeqIO
from tempfile import mkdtemp
from itertools import combinations
from collections import defaultdict, Counter
from matplotlib.patches import Rectangle


import json
import re
import Bio
import shutil
import logging
import importlib
import statistics
import subprocess
import numpy as np
import pandas as pd
import seaborn as sns
import networkx as nx
import plotly.express as px
import matplotlib.pyplot as plt
import plotly.graph_objects as go


In [ ]:
# nb6 comb_greedy_c0.9_ts10_mo3_mi0.9_mm10
# nb3 comb_greedy_c0.92_ts10_mo3_mi0.9_mm10
#nb13 comb_greedy_c0.86_ts10_mo3_mi0.8_mm12
# bd17 comb_greedy_c0.92_ts10_mo3_mi0.6_mm14

# best contig greedy results
# ma1 heavy comb_greedy_c0.92_ts10_mo3_mi0.9_mm12
# ma2 heavy comb_greedy_c0.86_ts10_mo3_mi0.8_mm14
# ma3 heavy comb_greedy_c0.9_ts10_mo3_mi0.6_mm12

# ma1 light comb_greedy_c0.88_ts10_mo3_mi0.9_mm10
# ma2 light comb_greedy_c0.9_ts10_mo3_mi0.9_mm10
# ma3 light comb_greedy_c0.92_ts10_mo3_mi0.9_mm8

In [ ]:
ass_method = 'greedy'
conf = 0.92
size_threshold = 10
min_overlap = 3
min_identity = 0.6
max_mismatches = 14

In [ ]:
comb = "comb_greedy_c0.92_ts10_mo3_mi0.6_mm14"

In [ ]:
params = {
        "ass_method": 'greedy',
        "conf": conf,
        "size_threshold": size_threshold,
        "min_overlap": min_overlap,
        "max_mismatches": max_mismatches,
        "min_identity": min_identity
    }

In [ ]:
# run = "bsa"
# protein = 'MKWVTFISLLLLFSSAYSRGVFRRDTHKSEIAHRFKDLGEEHFKGLVLIAFSQYLQQCPFDEHVKLVNELTEFAKTCVADESHAGCEKSLHTLFGDELCKVASLRETYGDMADCCEKQEPERNECFLSHKDDSPDLPKLKPDPNTLCDEFKADEKKFWGKYLYEIARRHPYFYAPELLYYANKYNGVFQECCQAEDKGACLLPKIETMREKVLASSARQRLRCASIQKFGERALKAWSVARLSQKFPKAEFVEVTKLVTDLTKVHKECCHGDLLECADDRADLAKYICDNQDTISSKLKECCDKPLLEKSHCIAEVEKDAIPENLPPLTADFAEDKDVCKNYQEAKDAFLGSFLYEYSRRHPEYAVSVLLRLAKEYEATLEECCAKDDPHACYSTVFDKLKHLVDEPQNLIKQNCDQFEKLGEYGFQNALIVRYTRKVPQVSTPTLVEVSRSLGKVGTRCCTKPESERMPCTEDYLSLILNRLCVLHEKTPVSEKVTKCCTESLVNRRPCFSALTPDETYVPKAFDEKLFTFHADICTLPDTEKQIKKQTALVELLKHKPKATEEQLKTVMENFVAFVDKCCAADDKEACFAVEGPKLVVSTQTALA'
# chain = ''
# proteases = ['Chymotrypsin', 'Legumain', 'Krakatoa', 'Elastase', 'Trypsin', 'Papain', 'Thermo', 'ProtK', 'GluC', 'LysC']

#run = "ma2"
#protein = "EVQLVQSGAEVKKPGSSVKVSCKASGGTFSSYALSWVRQAPGQGLEWMGGLLPLFGTANYAQKFQGRVTLTADESTSTAYMELRSLRSDDTAVYYCARDNLGYCSGGSCYSDYYYYYMDVWGQGTLVTVSSASTKGPSVFPLAPSSKSTSGGTAALGCLVKDYFPEPVTVSWNSGALTSGVHTFPAVLQSSGLYSLSSVVTVPSSSLGTQTYLCNVNHKPSNTKVDKKVEPKSCDKTHTCPPCPAPELLGGPSVFLFPPKPKDTLMLSRTPEVTCVVVDVSHEDPEVKFNWYVDGVEVHNAKTKPREEQYNSTYRVVSVLTVLHQDWLNGKEYKCKVSNKALPAPLEKTLSKAKGQPREPQVYTLPPSRDELTKNQVSLTCLVKGFYPSDLAVEWESNGQPENNYKTTPPVLDSDGSFFLYSKLTVDKSRWQQGNVFSCSVMHEALHNHYTQKSLSLSPGK"
#chain = "heavy"
#proteases = ["Thermo", "Papain", "Chemo", "Trypsin", "Elastase", "ProtK", "GluC"]

#run = "NB6"
#protein = 'QVQLQESGGGLVQPGGSLRLSCTASLNIFSINAMGWYRQAPGKQRELVAAITSGGSTNYADSVKGRFTISRDNAKSTVYLQMNSLKPEDTAVYYCHAEGPFNIATKEQYDYWGQGTQVTVSSAAADYKDHDGDYKDHDIDYKDDDDKGAAHHHHHH'
#proteases = ["Vesuvius", "Krakatoa", "Elastase", "Trypsin", "GluC", "Chymotrypsin", "Papain", "ProteinaseK", "Thermolysin"]
#chain = ""

#run = "NB3"
#protein = 'QVQLQESGGGLVQAGGSLRLSCLASGRTFSDYRIGWFRQAPGKEREFVSTIRNDDANTYYADSVKGRFTISRDNAKNTVYLQMNSLKPEDTAVYYCAAGARHTAQTMAAGKGIDYWGQGTQVTVSSAAADYKDHDGDYKDHDIDYKDDDDKGAAHHHHHH'
#proteases = ["Vesuvius", "Krakatoa", "Elastase", "Trypsin", "GluC", "Chymotrypsin", "Papain", "ProteinaseK", "Thermolysin"]
#chain = ""

run = "NB13"
protein = 'QVQLQESGGGLVQPGGSLRLSCAASGSASSMYTLAWYRQAPGKQRELVALITSGHMTHYEDSVKGRFTISRDNAKEVLYLQMNSLKPEDTAVYFCNLHRLTSSDDDGRTWGQGTQVTVSSAAADYKDHDGDYKDHDIDYKDDDDKGAAHHHHHH'
proteases = ["Vesuvius", "Krakatoa", "Elastase", "Trypsin", "GluC", "Chymotrypsin", "Papain", "ProteinaseK", "Thermolysin"]
chain = ""

In [ ]:
# run = "BIND17"
# protein = "KEELRAAAAELLAAAEALAEELRRLGLEEAAAHVLAAARHVAAALELIAATPASELNPELKREVAAHLREAAAHFEAAAEIVAAEDPLAGAMLREAALAARSMAAYVLHSSPEEALQQAAVFATGLAGAMLTMTGLVRERLAARGLNDIFEAQKIEWHEHHHHHH"
# proteases = ["Vesuvius","Chymo","GluC","Papain","Krakatoa","ProtK","Thermo","Trypsin","Elastase"]
# chain = ""

In [ ]:
folder_outputs = f"../outputs/{run}{chain}"

prep.create_directory(folder_outputs)

combination_folder_out = os.path.join(folder_outputs, f"comb_{ass_method}_c{conf}_ts{size_threshold}_mo{min_overlap}_mi{min_identity}_mm{max_mismatches}")

prep.create_subdirectories_outputs(combination_folder_out)


### Data cleaning

In [ ]:
protein_norm = prep.normalize_sequence(protein)

df = pd.read_csv(f"../input/{run}.csv")

In [ ]:
df['protease'] = df['experiment_name'].apply(lambda name: prep.extract_protease(name, proteases))

df = prep.clean_dataframe(df)

In [ ]:
df['cleaned_preds'] = df['preds'].apply(prep.remove_modifications)

In [ ]:
cleaned_psms = df['cleaned_preds'].tolist()

In [ ]:
filtered_psms = prep.filter_contaminants(cleaned_psms, run, "../fasta/contaminants.fasta")

In [ ]:
df = df[df['cleaned_preds'].isin(filtered_psms)]

In [ ]:
df["mapped"] = df["cleaned_preds"].apply(lambda x: "True" if x in protein_norm else "False")

In [ ]:
df = df[df['conf'] > conf]

In [ ]:
df.reset_index(drop=True, inplace=True)

In [ ]:
final_psms = df['cleaned_preds'].tolist()

In [ ]:
mapped_psms = map.process_protein_contigs_scaffold(final_psms, protein_norm, max_mismatches, min_identity)

In [ ]:
cdrs = {"CDR1": (31, 35), "CDR2": (50, 66), "CDR3": (99, 115)}
highlight_colors = {"CDR1": "orange", "CDR2": "lightgreen", "CDR3": "deepskyblue"}

depth = np.zeros(len(protein_norm), dtype=int)
for _, (start, end, _, _) in mapped_psms:
    depth[start:end] += 1

fig = go.Figure()
fig.add_trace(go.Scatter(
    x=list(range(len(protein_norm))),
    y=depth,
    mode='lines',
    line=dict(color='steelblue', width=2),
    fill='tozeroy',
    fillcolor='rgba(70, 130, 180, 0.2)', 
    name='PSM Depth'
))

for label, (start, end) in cdrs.items():
    fig.add_shape(
        type="rect",
        x0=start, x1=end,
        y0=0, y1=max(depth),
        fillcolor=highlight_colors[label],
        opacity=0.3,
        line=dict(width=0),
        layer="below"
    )
    fig.add_annotation(
        x=(start + end) / 2,
        y=max(depth) + 8,
        text=label,
        showarrow=False,
        font=dict(size=14, color="black"),
        xanchor="center"
    )

fig.update_layout(
    title='PSM depth across the protein sequence with CDR regions',
    xaxis=dict(
        title='Amino acid position',
        tickmode='linear',
        dtick=10,
        showline=True,
        linecolor='black',
        linewidth=2,
        showgrid=False
    ),
    yaxis=dict(
        title='Depth (Number of matching PSMs per position)',
        showline=True,
        linecolor='black',
        linewidth=2,
        showgrid=False
    ),
    template='plotly_white',
    height=450,
    width=1000,
    margin=dict(t=60),
    showlegend=False
)

fig.show()
fig.write_image('fig_4C_depth_psms_cdrs.svg', format='svg', scale=2)

### Assembly

In [ ]:
assembled_contigs = greedy.assemble_contigs(final_psms, min_overlap)

In [ ]:
assembled_contigs = list(set(assembled_contigs))

In [ ]:
assembled_contigs = [contig for contig in assembled_contigs if len(contig) > size_threshold]

In [ ]:
assembled_contigs = sorted(assembled_contigs, key=len, reverse=True)

In [ ]:
records = [Bio.SeqRecord.SeqRecord(Bio.Seq.Seq(contig), id=f"contig_{idx+1}",
                                    description=f"length: {len(contig)}") for idx,
                                    contig in enumerate(assembled_contigs)
                                    ]

In [ ]:
Bio.SeqIO.write(records, f"{combination_folder_out}/contigs/{ass_method}_contig_{conf}_{run}.fasta", "fasta")

In [ ]:
mapped_contigs = map.process_protein_contigs_scaffold(assembled_contigs, protein_norm, max_mismatches, min_identity)

In [ ]:
df_contigs_mapped = map.create_dataframe_from_mapped_sequences(data = mapped_contigs)

In [ ]:
comp_stat.compute_assembly_statistics(df = df_contigs_mapped, sequence_type='contigs', output_folder = f"{combination_folder_out}/statistics", reference = protein_norm, **params)

In [ ]:
map.mapping_substitutions(
    mapped_sequences = mapped_contigs,
    prot_seq = protein_norm,
    title=f"Contig mapping to reference sequence, {run}",
    contig_colors="#D8D9E8",
    match_color="#D8D9E8",
    output_file= f"fig_X_{run}_substitution_map_contigs_greedy.svg",
    output_folder="."
)

In [ ]:
def scaffold_iterative_greedy(contigs, min_overlap, size_threshold, disable_tqdm=False):
    prev = None
    current = contigs

    while prev != current:
        prev = current
        
        current = greedy.combine_seqs_into_scaffolds(current, min_overlap)
        current = list(set(current))
        current = [s for s in current if len(s) > size_threshold]
        current = sorted(current, key=len, reverse=True)

        current = greedy.combine_seqs_into_scaffolds(current, min_overlap)
        current = list(set(current))
        current = [s for s in current if len(s) > size_threshold]
        current = sorted(current, key=len, reverse=True)

        current = greedy.merge_contigs(current)
        current = list(set(current))
        current = [s for s in current if len(s) > size_threshold]
        current = sorted(current, key=len, reverse=True)

    return current

In [ ]:
assembled_scaffolds = scaffold_iterative_greedy(assembled_contigs, min_overlap, size_threshold)

In [ ]:
records = []

for i, seq in enumerate(assembled_scaffolds):
    record = Bio.SeqRecord.SeqRecord(Bio.Seq.Seq(seq), id=f"scaffold_{i+1}", description=f"length: {len(seq)}")
    records.append(record)

In [ ]:
Bio.SeqIO.write(records, f"{combination_folder_out}/scaffolds/{ass_method}_scaffold_{conf}_{run}.fasta", "fasta")

In [ ]:
mapped_scaffolds = map.process_protein_contigs_scaffold(assembled_scaffolds, protein_norm, max_mismatches, min_identity)

In [ ]:
df_scaffolds_mapped = map.create_dataframe_from_mapped_sequences(data = mapped_scaffolds)

In [ ]:
comp_stat.compute_assembly_statistics(df = df_scaffolds_mapped, sequence_type='scaffolds', output_folder = f"{combination_folder_out}/statistics", reference = protein_norm, **params)

In [ ]:
map.mapping_substitutions(mapped_sequences = mapped_scaffolds,
                          prot_seq = protein_norm,
                          title=f"Scaffold mapping to reference sequence, {run}",
                          contig_colors="#BCBDD9",
                          match_color="#BCBDD9", 
                          output_file=f"fig_X_{run}_substitution_map_scaffolds_greedy.svg",
                          output_folder="."
                          )


### Clustering

In [ ]:
scaffolds_folder_out = f"../outputs/{run}{chain}/{comb}/scaffolds"
print(f"scaffolds_folder_out: {scaffolds_folder_out}")

In [ ]:
importlib.reload(cons)

clus.cluster_fasta_files(input_folder = scaffolds_folder_out)

In [ ]:
cluster_folder_out = os.path.join(scaffolds_folder_out, "cluster")
print(cluster_folder_out)

In [ ]:
cluster_tsv_folder = os.path.join(scaffolds_folder_out, "cluster")
output_base_folder = os.path.join(scaffolds_folder_out, "cluster_fasta")

for fasta_file in os.listdir(scaffolds_folder_out):
    if fasta_file.endswith('.fasta'):
        fasta_path = os.path.join(scaffolds_folder_out, fasta_file)
        clus.process_fasta_and_clusters(fasta_path, cluster_tsv_folder, output_base_folder)

### Alignment

In [ ]:
cluster_fasta_folder = os.path.join(scaffolds_folder_out, "cluster_fasta") 
align_folder = os.path.join(scaffolds_folder_out, "align")
prep.create_directory(align_folder)

In [ ]:
def align_or_copy_fasta(fasta_file, output_file):

    sequences = list(SeqIO.parse(fasta_file, "fasta"))
    
    if len(sequences) == 1:
        shutil.copy(fasta_file, output_file)
    else:
        subprocess.run(["clustalo", "-i", fasta_file, "-o", output_file, "--outfmt", "fa"])


In [ ]:
for cluster_folder in os.listdir(cluster_fasta_folder): 
    cluster_folder_path = os.path.join(cluster_fasta_folder, cluster_folder) 
    if os.path.isdir(cluster_folder_path): 
        
        output_cluster_folder = os.path.join(align_folder, cluster_folder) 
        os.makedirs(output_cluster_folder, exist_ok=True) 
            
        for fasta_file in os.listdir(cluster_folder_path): 
            if fasta_file.endswith('.fasta'): 
                fasta_file_path = os.path.join(cluster_folder_path, fasta_file)
                base_filename = os.path.splitext(fasta_file)[0] 
                output_file = os.path.join(output_cluster_folder, f"{base_filename}_out.afa")
                    
                align_or_copy_fasta(fasta_file_path, output_file)

print("All alignment tasks completed.")

### Consensus

In [ ]:
consensus_folder = os.path.join(scaffolds_folder_out, "consensus")

In [ ]:
importlib.reload(cons)

cons.process_alignment_files(align_folder, consensus_folder)

In [ ]:
all_sequences = cons.load_all_consensus_sequences(consensus_folder)

## CDR regions analysis

In [ ]:
def plot_cdr_scaffolds(reference_seq, scaffold_info, cdrs, highlight_colors,
                       font_family='monospace', font_size=25, letter_spacing=1.5, save_path=None):
    """
    Plot reference sequence and full scaffold sequences aligned, highlighting CDR regions,
    with adjustable letter spacing for sequence letters.
    
    """
    # Helper for L/I equivalence
    def chars_equal(a, b):
        return (a in ['L','I'] and b in ['L','I']) or a == b

    # Find offset where seq aligns in ref
    def find_alignment_offset(ref, seq):
        for pos in range(len(ref) - len(seq) + 1):
            if all(chars_equal(ref[pos+i], seq[i]) for i in range(len(seq))):
                return pos
        return 0

    # Dynamically adjust figure width to letter spacing and sequence length
    fig_width = max(10, len(reference_seq) * letter_spacing / 8)
    fig, ax = plt.subplots(figsize=(fig_width, 6))
    ax.axis('off')

    # Plot CDR highlights on reference
    for name, (start, end) in cdrs.items():
        x0 = (start - 1) * letter_spacing + 1
        width = (end - start + 1) * letter_spacing
        ax.add_patch(Rectangle((x0, 4.5), width, 0.8, color=highlight_colors[name], alpha=0.3))
        ax.text(x0 + width/2, 5.5, name, ha='center', va='bottom',
                fontsize=font_size, fontweight='bold', color=highlight_colors[name])

    # Plot reference sequence with spaced letters
    ax.text(-5, 5, "Reference", fontfamily=font_family, fontsize=font_size, ha='right')
    for i, aa in enumerate(reference_seq):
        x = i * letter_spacing + 1
        ax.text(x, 5, aa, fontfamily=font_family, fontsize=font_size, color='black')

    y_positions = [4, 3, 2]
    for idx, (scaf_name, seq) in enumerate(scaffold_info):
        cdr_name = f"CDR{idx+1}"
        color = highlight_colors[cdr_name]
        offset = find_alignment_offset(reference_seq, seq)

        # Calculate relative CDR positions
        start, end = cdrs[cdr_name]
        rel_start = max(start - 1 - offset, 0)
        rel_end = min(end - 1 - offset, len(seq) - 1)

        # Label including scaffold number
        ax.text(-5, y_positions[idx], f"{cdr_name} ({scaf_name})",
                fontfamily=font_family, fontsize=font_size, ha='right')

        # Draw each letter with spacing
        for i, aa in enumerate(seq):
            x = (offset + i) * letter_spacing + 1
            ax.text(x, y_positions[idx], aa, fontfamily=font_family,
                    fontsize=font_size, color='black')

        # Highlight CDR region on scaffold
        if rel_end >= rel_start:
            x0 = (offset + rel_start) * letter_spacing + 1
            width = (rel_end - rel_start + 1) * letter_spacing
            ax.add_patch(Rectangle((x0, y_positions[idx] - 0.2),
                                   width, 0.8, color=color, alpha=0.3))

    ax.set_xlim(0, len(reference_seq) * letter_spacing + 5)
    ax.set_ylim(1.5, 6)

    plt.tight_layout()
    if save_path:
        plt.savefig(save_path, format='svg', dpi=600, bbox_inches='tight')
    plt.show()

reference_seq = ("QVQLQESGGGLVQPGGSLRLSCAASGSASSMYTLAWYRQAPGKQRELVALITSGHMTHYEDSVKGRFTISRDNAKEVLYLQMNSLKPEDTAVYFCNLHRLTSSDDDGRTWGQGTQVTVSSAAADYKDHDGDYKDHDIDYKDDDDKGAAHHHHHH")


cdrs = {"CDR1": (31, 35), "CDR2": (50, 66), "CDR3": (99, 115)}
highlight_colors = {"CDR1": "orange", "CDR2": "lightgreen", "CDR3": "deepskyblue"}
scaffold_info =[
    ("scaffold_5", "QVQLQESGGGLVQPGGSLRLSCAASGSASSMYTLAWYRQAPGKQRELVALLTSGHMTHYEDSVKGRFY"),
    ("scaffold_10", "SYFCNLHRLTSSDDDGRTWGQGTQVTVSSAAADYKDHDGDYKDHDLDYKDDDDKGAAH")
]

plot_cdr_scaffolds(
    reference_seq,
    scaffold_info,
    cdrs,
    highlight_colors,
    font_size=20,
    letter_spacing=3,  # increased spacing
    save_path="cdrs_scaffolds.svg"
)
